# Tennisspielen bei verschiedenem Wetter

Für Aufgaben, die typisch für Excel wären, kann man in Python sehr gut auf die Bibliothek pandas zurückgreifen.

In [ ]:
import pandas as pd

In Jupyter Notebooks ist das Fragezeichen als Operator dafür da, um Hilfe anzuzeigen.
Dies funktioniert für Module, aber auch für Datentypen etc.
Außerdem lässt sich für alles über google immer schnell der entsprechende Handbucheintrag finden.
Ebenfalls ist stackoverflow eine beliebte Informationsquelle.

In [ ]:
?pd

In [ ]:
df = pd.read_csv("tennis.tsv", sep=" \t", engine="python")
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
other_columns = set(df.columns) - set(["Day"])
other_columns

In [ ]:
for column in other_columns:
    print()
    print("Attribut: ", column, "\n")
    print(df[column].value_counts())
    print()

Es wird häufiger Tennis gespielt als nicht gespielt.
Daraus können wir uns nun ein dummes Vorhersagewerkzeug basteln.
Es wird einfach getippt, dass der Nachbar immer Tennis spielen geht!

In [ ]:
df = df.assign(stupid_guess=["Yes" for _ in range(len(df))])

df

## Metriken

Wie gut ist denn nun das dumme Vorhersagewerkzeug?
Wie kann man das messen?
Auf https://de.wikipedia.org/wiki/Beurteilung_eines_bin%C3%A4ren_Klassifikators findet man zu dem Thema Hilfe.
In diesem Beispiel ist die Treffergenauigkeit geeignet.
Diese drückt aus, in wie viel Prozent der Fälle die Klassifikation richtig war.

Das müssen wir nicht selbst implementieren, dafür gibt es bereits eine Bibliothek in Python.

In [ ]:
import sklearn.metrics

In [ ]:
sklearn.metrics.accuracy_score(df["Play Tennis?"], df["stupid_guess"])

In 64 % der Fälle liegen wir also damit richtig.

Wie würde es aussehen, wenn wir immer eine Münze werfen?
Dafür gibt es ein weiteres Modul.

In [ ]:
import random

In [ ]:
def random_guess():
    return ["Yes" if random.random() > .5 else "No" 
            for _ in range(len(df))]

random_guess()

In [ ]:
for i in range(10):
    this_guess = random_guess()
    accuracy = sklearn.metrics.accuracy_score(df["Play Tennis?"], this_guess)
    print(f"{i+1:02}) Zufälliges Ergebnis: {accuracy}")

## Verwende einen Entscheidungsbaum

In [ ]:
import sklearn.tree

In [ ]:
dt = sklearn.tree.DecisionTreeClassifier()

In [ ]:
?dt

Die Eingabe 

In [ ]:
eingabe = df[["Day", "Outlook", "Temperature", "Humidity", "Wind"]]
eingabe

In [ ]:
ziel = df[["Play Tennis?"]]
ziel

Nun wird der Zusammenhang zwischen Eingabe und der Beobachtung gelernt

In [ ]:
dt.fit(eingabe, ziel)

Die letzte Zelle sollte zum Fehler `ValueError: could not convert string to float: 'Sunny'` geführt haben.
Hier müssen wir für die Methode die Daten vorverarbeiten.

Eine Möglichkeit ist das sogenannte One-Hot-Enkodieren.
Dafür werden alle vorhandenen Kategorien eines Attributs als eine eigene Spalte aufgenommen.
Diese Fähigkeit bringt das Modul pandas bereits mit.

In [ ]:
one_hot_data = pd.get_dummies(df[["Day", "Outlook", "Temperature", "Humidity", "Wind"]])

one_hot_data

Tatsächlich arbeteitet die Bibliothek mit der Matrix, die hinter der oben sichtbaren Tabelle liegt.

In [ ]:
one_hot_data.values

In [ ]:
dt.fit(one_hot_data, ziel)

In [ ]:
y_pred = dt.predict(one_hot_data)
y_pred

In [ ]:
sklearn.metrics.accuracy_score(df["Play Tennis?"], y_pred)

Uuups, das ist aber ganz schön gut.
Kann man den Nachbarn wirklich so leicht durchschauen?

In [ ]:
one_hot_data.iloc[0]

Für Weiterdenkende:
Auf https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html wird Schritt für Schritt erklärt, wie man die Entscheidung eines Entscheidungsbaums nachvollziehen kann.